In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-06-30 07:35:51.857748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 07:35:51.969302: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 07:35:52.530751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [3]:
def chunks(lst, n):
    #sort the list
    lst.sort()
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


N real and fake phrases: 500, 500
492 491
/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX154/real/MRLD0_SX154.WAV
# of Train instances: 589
# of Dev instances: 196
# of Test instances: 198


In [5]:
len(real_speakers + fake_speakers)

352

In [6]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MRLD0_SX154.WAV'

In [7]:
#df

In [8]:
'''
import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)
'''

'\nimport os\nimport shutil\n\nfile_paths = list(df[\'orig_path\'])\noutput_dir = \'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original\'\n\ndef copy_files_to_directory(file_paths, destination_directory):\n    for file_path in file_paths:\n        # Extract the file name from the path\n        file_name = os.path.basename(file_path)\n        \n        # Create the destination path by joining the destination directory and the file name\n        destination_path = os.path.join(destination_directory, file_name)\n        \n        try:\n            # Copy the file to the destination directory\n            shutil.copy2(file_path, destination_path)\n            print(f"Successfully copied {file_path} to {destination_path}")\n        except FileNotFoundError:\n            print(f"File {file_path} not found.")\n        except shutil.Error as e:\n            print(f"Error occurred while copying {file_path}: {e}")\n            \ncopy_files_to_directory(file_paths, output_dir)\n'

In [9]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX163/fake/Rachel_SX163.wav'

In [10]:
#df[df['label'] == 1]

In [11]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [12]:
### End of timit additions ###
df[(df['type']=='train') | (df['type']=='dev')].value_counts('multiclass_label')

multiclass_label
0    397
1    388
dtype: int64

In [13]:
df

,type,id,architecture,orig_path,label,multiclass_label,path,speaker
0,train,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRLD0
1,test,1,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MDLC1
2,test,2,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MMSM0
3,dev,3,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MTJG0
4,test,4,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,0,0,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/M...,MRCW0
...,...,...,...,...,...,...,...,...
978,dev,978,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/B...,Biden
979,dev,979,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/D...,Domi
980,train,980,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/D...,Domi
981,train,981,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT a...,1,1,/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/J...,Josh


In [14]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=10)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [15]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [16]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    
    #if data_df is None:
        #data_df = loader.generate_split_speaker_test(speakers_to_remove)
        #'/home/ubuntu/data/results/TIMIT/testing/test5.csv'
        #data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/testing/{name}.csv')
    
    data_df = loader.generate_split_speaker_test(speakers_to_remove)
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [19]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
results = run(['ThisDoesNotMatter'], file_path, 'test7', loader=timit_data_loader)

# Multiple results run
#results = []
counter = 0 

"""for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(fake_speaker_chunk)
        print(real_speaker_chunk)
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()
        #if counter == 1:
        #    break
    #if counter == 1:
    #    break
"""
        


N real and fake phrases: 500, 500
492 491
/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX154/real/MRLD0_SX154.WAV
# of Train instances: 514
# of Dev instances: 171
# of Test instances: 125
[NeMo I 2023-06-30 07:38:13 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 07:38:13 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 07:38:13 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 07:38:14 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 07:38:14 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 07:38:14 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 07:38:14 features:267] PADDING: 16
[NeMo I 2023-06-30 07:38:15 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. pcm_fftMag_spectralEntropy_sma_upleveltime50
2. pcm_fftMag_spectralEntropy_sma_de_minPos
3. pcm_fftMag_spectralEntropy_sma_de_stddev
4. audSpec_Rfilt_sma[18]_centroid
5. pcm_fftMag_spectralEntropy_sma_range
6. pcm_fftMag_spectralVariance_sma_lpgain
7. pcm_fftMag_spectralEntropy_sma_de_lpgain
8. pcm_fftMag_spectralEntropy_sma_kurtosis
9. pcm_RMSenergy_sma_minPos
10. audSpec_Rfilt_sma[4]_centroid
11. pcm_fftMag_spectralEntropy_sma_de_lpc0
12. audSpec_Rfilt_sma[14]_centroid
13. audSpec_Rfilt_sma[19]_centroid
14. pcm_fftMag_spectralEntropy_sma_de_percentile99.0
15. pcm_fftMag_spectralEntropy_sma_lpgain
16. pcm_fftMag_spectralEntropy_sma_linregerrQ
17. pcm_fftMag_spectralEntropy_sma_skewness
18. pcm_fftMag_spectralRollOff50.0_sma_quartile1
19. audspec_lengthL1norm_sma_minPos
20. pcm_fftMag_spectralEntropy_sma_de_lpc4
Initializing data...
Standardizing features...
smileFea

[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/ipykernel_4048925/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/ipykernel_4048925/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/i


Selected features:.

1. pcm_fftMag_spectralEntropy_sma_upleveltime50
2. pcm_fftMag_spectralEntropy_sma_de_minPos
3. pcm_fftMag_spectralEntropy_sma_de_stddev
4. audSpec_Rfilt_sma[18]_centroid
5. pcm_fftMag_spectralEntropy_sma_range
6. pcm_fftMag_spectralVariance_sma_lpgain
7. pcm_fftMag_spectralEntropy_sma_de_lpgain
8. pcm_fftMag_spectralEntropy_sma_kurtosis
9. pcm_RMSenergy_sma_minPos
10. audSpec_Rfilt_sma[4]_centroid
11. pcm_fftMag_spectralEntropy_sma_de_lpc0
12. audSpec_Rfilt_sma[14]_centroid
13. audSpec_Rfilt_sma[19]_centroid
14. pcm_fftMag_spectralEntropy_sma_de_percentile99.0
15. pcm_fftMag_spectralEntropy_sma_lpgain
16. pcm_fftMag_spectralEntropy_sma_linregerrQ
17. pcm_fftMag_spectralEntropy_sma_skewness
18. pcm_fftMag_spectralRollOff50.0_sma_quartile1
19. audspec_lengthL1norm_sma_minPos
20. pcm_fftMag_spectralEntropy_sma_de_lpc4


[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/ipykernel_4048925/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/ipykernel_4048925/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 07:38:21 nemo_logging:349] /tmp/i

"for fake_speaker_chunk in fake_speaker_partitions:\n    for real_speaker_chunk in real_speaker_partitions:\n        print(fake_speaker_chunk)\n        print(real_speaker_chunk)\n        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')\n        voices_to_remove = fake_speaker_chunk+real_speaker_chunk\n        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))\n        counter += 1\n        clear_output()\n        #if counter == 1:\n        #    break\n    #if counter == 1:\n    #    break\n"

In [20]:
results

,feature_method,model,fake_cols,label_type,acc,cls_acc,loss,eer_score,eer_threshold
0,titanet,logreg,[ThisDoesNotMatter],label,1.000,"{0: 1.0, 1: 1.0}",0.322456,0.000000,0.599857
0,openSmile_binary,logreg,[ThisDoesNotMatter],label,0.768,"{0: 0.0, 1: 1.0}",0.403752,0.114583,0.691826
0,openSmile_multiclass,logreg,[ThisDoesNotMatter],label,0.768,"{0: 0.0, 1: 1.0}",0.403752,0.114583,0.691826
0,cadence,logreg,[ThisDoesNotMatter],label,0.864,"{0: 0.7241379310344828, 1: 0.90625}",0.340157,0.166667,0.583781
0,titanet,logreg,[ThisDoesNotMatter],multiclass_label,1.000,"{0: 1.0, 1: 1.0}",0.322456,NaN,NaN
0,openSmile_binary,logreg,[ThisDoesNotMatter],multiclass_label,0.768,"{0: 0.0, 1: 1.0}",0.403752,NaN,NaN
0,openSmile_multiclass,logreg,[ThisDoesNotMatter],multiclass_label,0.768,"{0: 0.0, 1: 1.0}",0.403752,NaN,NaN
0,cadence,logreg,[ThisDoesNotMatter],multiclass_label,0.864,"{0: 0.7241379310344828, 1: 0.90625}",0.340157,NaN,NaN
0,titanet,random_forest,[ThisDoesNotMatter],label,0.944,"{0: 1.0, 1: 0.9270833333333334}",0.341357,0.000000,0.380000
0,openSmile_binary,random_forest,[ThisDoesNotMatter],label,0.992,"{0: 1.0, 1: 0.9895833333333334}",0.291067,0.010417,0.950000


In [18]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/testing/test6.csv')

NameError: name 'results' is not defined